# Metadata

**L1 Taxonomy** - SQL in Python

**L2 Taxonomy** - Data Structures

**Subtopic** - Implementing a basic hash table with linear probing or chaining for collisions

**Use Case** - Implement a Python module that constructs a basic hash table. The hash table should be able to handle collisions using either linear probing or chaining methods. The module should allow for adding, removing, and retrieving key-value pairs from the hash table. Ensure the hash table can handle string keys and any Python data type as values. Finally, the module should be able to resize itself when it reaches a certain load factor to maintain efficient operations.

**Programming Language** - Python

**Target Model** - o1

# Setup

```requirements.txt
typing-extensions==4.9.0
mypy==1.8.0
```


# Prompt

###Problem Description:

You have been tasked with implementing a basic hash table in Python which will have the capacity to support two types of collision resolution: `chaining` and `linear probing`. The hash table should be able to dynamically resize itself when the load factor exceeds a defined threshold.
The task here is to implement a HashTable class with the following core methods:
- `put(key: str, value: Any) -> None`
- `get(key: str) -> Any`
- `remove(key: str) -> None`

The user should  be able to specify the collision resolution strategy at initialization or at the beginning which could be any of `'chaining'` or `'linear_probing'`, initial capacity, and load factor threshold.

###Input Format:

```
ht = HashTable(
    collision_resolution='chaining',
    initial_capacity=8,
    load_factor_threshold=0.75
)

ht.put("apple", 10)
ht.get("apple")
ht.remove("apple")

```

###Input validation and Error Handling messages:

- When the key is not a non-empty string, raise `TypeError` with the message: `Key must be a non-empty string`
- When the collision resolution type is not `'chaining'` or `'linear_probing'`, raise `ValueError` with the message: `Collision resolution must be 'chaining' or 'linear_probing'`
- When attempting to insert a duplicate key, raise `KeyError` with the message: `Key 'apple' already exists`
- When attempting to retrieve or remove a key that does not exist, raise `KeyError` with the message: `Key 'banana' not found`
- When the initial capacity is not a positive integer, raise `ValueError` with the message: `Initial capacity must be a positive integer`
- When the load factor threshold is not a float between 0 and 1 (exclusive), raise `ValueError` with the message: `Load factor threshold must be a float between 0 and 1 (exclusive)`

###Output Format:

- Return the appropriate value (`get`)
- Raise appropriate exceptions on invalid operations (e.g., missing key)

###Input and Output Example 1:
```
ht = HashTable(collision_resolution='chaining')
ht.put("dog", 5)
print(ht.get("dog"))          # Output: 5
ht.remove("dog")
ht.get("dog")                 # Raises KeyError

```
###Input and Output Example 2:
```
ht = HashTable(collision_resolution='linear_probing', initial_capacity=4)
ht.put("alpha", [1, 2, 3])
ht.put("beta", {"x": 42})
print(ht.get("beta"))         # Output: {"x": 42}
```


# Requirements

###Function Signature:
```
class HashTable:
    def __init__(self, collision_resolution: str = 'chaining', initial_capacity: int = 8, load_factor_threshold: float = 0.75):
        ...
    
    def put(self, key: str, value: Any) -> None:
        ...
    
    def get(self, key: str) -> Any:
        ...
    
    def remove(self, key: str) -> None:
        ...

```
###Implicit Requirements:

- When the table resizes, make sure to `rehash and move all existing entries` into the new structure correctly.
- Keys must always be `non-empty strings`, but it's okay to use `None` or any type as a `value`.
- iin the case of inserting an existing key raise the appropriate error.
- The hash table should be fast: ideally, inserting and looking up items should take constant time on average (O(1)).

###Explicit Requirements:

- Ensure to create a class called `HashTable` that supports the following methods:
  - `put(key: str, value: Any)`: Adds a new key-value pair to the table.
  - `get(key: str)`: Retrieves the value for a given key.
  - `remove(key: str)`: Removes a key and its value from the table.
- The table should support two collision handling methods:
  - `'chaining'`: this is using lists to handle multiple values in a slot.
  - `'linear_probing'`: this is searching the next available slot when there's a collision.
- The table should be able to resize itself automatically when the number of items reaches a certain threshold or load factor.
- You should also make sure to raise clear and appropriate error messages if:
  - The inputs are invalid.
  - The key does not exist during `get` or `remove`.
  - The configuration provided is incorrect.

###Edge Cases Behaviors:

It is expected that all the below edge cases and constraint should be handled efficiently.

- Inserting a key that causes the load factor to exceed the threshold should triggers resizing.
- Deleting from an empty or minimal or small size table should not crash.
- Attempting to use non-string keys should raises the appropriate error.
- Attempting to use unsupported collision method should raise appropriate error.


In [ ]:
# code
"""This module implements a HashTable with collision resolution.

using chaining or linear probing.
"""
from typing import Any


class HashTable:
    """HashTable DS with support for chaining and linear probing."""

    def __init__(
        self,
        collision_resolution: str = 'chaining',
        initial_capacity: int = 8,
        load_factor_threshold: float = 0.75
    ):
        """Initialize the HashTable with specified parameters."""
        if collision_resolution not in (
            'chaining', 'linear_probing'
                ):
            raise ValueError(
                "Collision resolution must be 'chaining' or 'linear_probing'"
                )

        if not isinstance(initial_capacity, int) or initial_capacity <= 0:
            raise ValueError("Initial capacity must be a positive integer")

        if not (0 < load_factor_threshold < 1):
            raise ValueError(
                "Load factor threshold must be a float "
                "between 0 and 1 (exclusive)"
                )

        self.collision_resolution = collision_resolution
        self.initial_capacity = initial_capacity
        self.capacity = initial_capacity
        self.load_factor_threshold = load_factor_threshold
        self.size = 0
        self._deleted_sentinel = object()
        self._deleted_count = 0

        if self.collision_resolution == 'chaining':
            self._table = [[] for _ in range(self.capacity)]
        else:
            self._table = [None] * self.capacity

    def put(self, key: str, value: Any) -> None:
        """Insert a key-value pair into the hash table."""
        if not isinstance(key, str) or not key:
            raise TypeError("Key must be a non-empty string")

        if self.collision_resolution == 'chaining':
            self._put_chaining(key, value)
        else:
            self._put_linear_probing(key, value)

        # Calculate current load factor
        cl = self.size / self.capacity
        current_load = (self.size + self._deleted_count) / self.capacity if \
            self.collision_resolution == 'linear_probing' else cl

        if current_load > self.load_factor_threshold:
            self._resize()

    def get(self, key: str) -> Any:
        """Retrieve a value by its key from the hash table."""
        if not isinstance(key, str) or not key:
            raise TypeError("Key must be a non-empty string")

        if self.collision_resolution == 'chaining':
            return self._get_chaining(key)
        return self._get_linear_probing(key)

    def remove(self, key: str) -> None:
        """Remove a key-value pair from the hash table."""
        if not isinstance(key, str) or not key:
            raise TypeError("Key must be a non-empty string")

        if self.collision_resolution == 'chaining':
            self._remove_chaining(key)
        else:
            self._remove_linear_probing(key)

    def _hash(self, key: str) -> int:
        """Compute the hash index for a given key."""
        return hash(key) % self.capacity

    def _put_chaining(self, key: str, value: Any) -> None:
        """Insert a key-value pair using chaining."""
        index = self._hash(key)
        bucket = self._table[index]

        for existing_key, _ in bucket:
            if existing_key == key:
                raise KeyError(f"Key '{key}' already exists")

        bucket.append((key, value))
        self.size += 1

    def _get_chaining(self, key: str) -> Any:
        """Retrieve a value using chaining."""
        index = self._hash(key)
        bucket = self._table[index]

        for existing_key, val in bucket:
            if existing_key == key:
                return val
        raise KeyError(f"Key '{key}' not found")

    def _remove_chaining(self, key: str) -> None:
        """Remove a key-value pair using chaining."""
        index = self._hash(key)
        bucket = self._table[index]

        for i, (existing_key, _) in enumerate(bucket):
            if existing_key == key:
                del bucket[i]
                self.size -= 1
                return
        raise KeyError(f"Key '{key}' not found")

    def _put_linear_probing(self, key: str, value: Any) -> None:
        """Insert a key-value pair using linear probing."""
        index = self._hash(key)
        first_deleted = None

        for _ in range(self.capacity):
            entry = self._table[index]

            if entry is None:
                if first_deleted is not None:
                    self._table[first_deleted] = (key, value)
                    self._deleted_count -= 1
                else:
                    self._table[index] = (key, value)
                self.size += 1
                return
            elif entry is self._deleted_sentinel:
                if first_deleted is None:
                    first_deleted = index
                index = (index + 1) % self.capacity
            else:
                existing_key, _ = entry
                if existing_key == key:
                    raise KeyError(f"Key '{key}' already exists")
                index = (index + 1) % self.capacity

        if first_deleted is not None:
            self._table[first_deleted] = (key, value)
            self._deleted_count -= 1
            self.size += 1
            return

        raise RuntimeError("Hash table is full")

    def _get_linear_probing(self, key: str) -> Any:
        """Retrieve a value using linear probing."""
        index = self._hash(key)

        for _ in range(self.capacity):
            entry = self._table[index]
            if entry is None:
                break
            elif entry is not self._deleted_sentinel:
                existing_key, val = entry
                if existing_key == key:
                    return val
            index = (index + 1) % self.capacity

        raise KeyError(f"Key '{key}' not found")

    def _remove_linear_probing(self, key: str) -> None:
        """Remove a key-value pair using linear probing."""
        index = self._hash(key)

        for _ in range(self.capacity):
            entry = self._table[index]
            if entry is None:
                break
            elif entry is not self._deleted_sentinel:
                existing_key, _ = entry
                if existing_key == key:
                    self._table[index] = self._deleted_sentinel
                    self.size -= 1
                    self._deleted_count += 1
                    return
            index = (index + 1) % self.capacity

        raise KeyError(f"Key '{key}' not found")

    def _resize(self) -> None:
        """Resize the hashtable when the load factor threshold is exceeded."""
        new_capacity = self.capacity * 2
        old_table = self._table

        if self.collision_resolution == 'chaining':
            self._table = [[] for _ in range(new_capacity)]
        else:
            self._table = [None] * new_capacity
            self._deleted_count = 0

        old_size = self.size
        self.size = 0
        self.capacity = new_capacity

        if self.collision_resolution == 'chaining':
            for bucket in old_table:
                for key, value in bucket:
                    self._put_chaining(key, value)
        else:
            for entry in old_table:
                if entry is not None and entry is not self._deleted_sentinel:
                    key, value = entry
                    self._put_linear_probing(key, value)

        self.size = old_size


In [ ]:
# tests
"""Test suite for the HashTable class."""
import unittest
from main import HashTable


class TestHashTable(unittest.TestCase):
    """Unit tests for HashTable."""

    def setUp(self):
        """Prepare hash tables for tests."""
        self.chaining_ht = HashTable(
            collision_resolution="chaining"
        )
        self.linear_ht = HashTable(
            collision_resolution="linear_probing"
        )

    def test_initialization_default(self):
        """Test default initialization."""
        ht = HashTable()
        self.assertEqual(ht.collision_resolution, "chaining")
        self.assertEqual(ht.capacity, 8)
        self.assertEqual(ht.load_factor_threshold, 0.75)

    def test_initialization_custom(self):
        """Test custom initialization."""
        ht = HashTable(
            collision_resolution="linear_probing",
            initial_capacity=16,
            load_factor_threshold=0.6
        )
        self.assertEqual(ht.collision_resolution, "linear_probing")
        self.assertEqual(ht.capacity, 16)
        self.assertEqual(ht.load_factor_threshold, 0.6)

    def test_invalid_collision_resolution(self):
        """Test invalid collision resolution."""
        with self.assertRaises(ValueError) as context:
            HashTable(collision_resolution="invalid_method")
        self.assertEqual(
            str(context.exception),
            "Collision resolution must be 'chaining' or 'linear_probing'"
        )

    def test_invalid_initial_capacity(self):
        """Test invalid initial capacity."""
        with self.assertRaises(ValueError) as context:
            HashTable(initial_capacity=0)
        self.assertEqual(
            str(context.exception),
            "Initial capacity must be a positive integer"
        )

    def test_invalid_load_factor_threshold(self):
        """Test invalid load factor threshold."""
        with self.assertRaises(ValueError) as context:
            HashTable(load_factor_threshold=1.5)
        self.assertEqual(
            str(context.exception),
            "Load factor threshold must be a float "
            "between 0 and 1 (exclusive)"
        )

    def test_put_and_get_chaining(self):
        """Test put and get with chaining."""
        self.chaining_ht.put("apple", 10)
        self.assertEqual(self.chaining_ht.get("apple"), 10)

    def test_put_and_get_linear_probing(self):
        """Test put and get with linear probing."""
        self.linear_ht.put("banana", 20)
        self.assertEqual(self.linear_ht.get("banana"), 20)

    def test_put_duplicate_key(self):
        """Test put with duplicate key."""
        self.chaining_ht.put("apple", 10)
        with self.assertRaises(KeyError) as context:
            self.chaining_ht.put("apple", 20)
        self.assertTrue(
            "Key 'apple' already exists" in str(context.exception)
        )

    def test_put_invalid_key(self):
        """Test put with invalid key."""
        with self.assertRaises(TypeError) as context:
            self.chaining_ht.put(123, "value")
        self.assertEqual(
            str(context.exception),
            "Key must be a non-empty string"
        )

    def test_get_nonexistent_key(self):
        """Test get with nonexistent key."""
        with self.assertRaises(KeyError) as context:
            self.chaining_ht.get("nonexistent")
        self.assertTrue(
            "Key 'nonexistent' not found" in str(context.exception)
        )

    def test_remove_key_chaining(self):
        """Test remove key with chaining."""
        self.chaining_ht.put("apple", 10)
        self.chaining_ht.remove("apple")
        with self.assertRaises(KeyError):
            self.chaining_ht.get("apple")

    def test_remove_key_linear_probing(self):
        """Test remove key with linear probing."""
        self.linear_ht.put("banana", 20)
        self.linear_ht.remove("banana")
        with self.assertRaises(KeyError):
            self.linear_ht.get("banana")

    def test_remove_nonexistent_key(self):
        """Test remove nonexistent key."""
        with self.assertRaises(KeyError) as context:
            self.chaining_ht.remove("nonexistent")
        self.assertTrue(
            "Key 'nonexistent' not found" in str(context.exception)
        )

    def test_resize_chaining(self):
        """Test resizing with chaining."""
        ht = HashTable(
            collision_resolution="chaining",
            initial_capacity=4,
            load_factor_threshold=0.6
        )
        ht.put("a", 1)
        ht.put("b", 2)
        ht.put("c", 3)
        self.assertGreater(ht.capacity, 4)

    def test_resize_linear_probing(self):
        """Test resizing with linear probing."""
        ht = HashTable(
            collision_resolution="linear_probing",
            initial_capacity=4,
            load_factor_threshold=0.6
        )
        ht.put("a", 1)
        ht.put("b", 2)
        ht.put("c", 3)
        self.assertGreater(ht.capacity, 4)

    def test_collision_chaining(self):
        """Test collision handling with chaining."""
        ht = HashTable(
            collision_resolution="chaining",
            initial_capacity=1
        )
        ht.put("a", 1)
        ht.put("b", 2)
        self.assertEqual(ht.get("a"), 1)
        self.assertEqual(ht.get("b"), 2)

    def test_collision_linear_probing(self):
        """Test collision handling with linear probing."""
        ht = HashTable(
            collision_resolution="linear_probing",
            initial_capacity=2
        )
        ht.put("a", 1)
        ht.put("b", 2)
        ht.put("c", 3)
        self.assertEqual(ht.get("a"), 1)
        self.assertEqual(ht.get("b"), 2)
        self.assertEqual(ht.get("c"), 3)

    def test_rehashing_chaining(self):
        """Test rehashing after resize with chaining."""
        ht = HashTable(
            collision_resolution="chaining",
            initial_capacity=2
        )
        ht.put("a", 1)
        ht.put("b", 2)
        ht.put("c", 3)
        self.assertEqual(ht.get("a"), 1)
        self.assertEqual(ht.get("b"), 2)
        self.assertEqual(ht.get("c"), 3)

    def test_rehashing_linear_probing(self):
        """Test rehashing after resize with linear probing."""
        ht = HashTable(
            collision_resolution="linear_probing",
            initial_capacity=2
        )
        ht.put("a", 1)
        ht.put("b", 2)
        ht.put("c", 3)
        self.assertEqual(ht.get("a"), 1)
        self.assertEqual(ht.get("b"), 2)
        self.assertEqual(ht.get("c"), 3)

    def test_empty_key(self):
        """Test put with empty key."""
        with self.assertRaises(TypeError) as context:
            self.chaining_ht.put("", "value")
        self.assertEqual(
            str(context.exception),
            "Key must be a non-empty string"
        )

    def test_reinsert_after_deletion_linear_probing(self):
        """Test reinsert after deletion with linear probing."""
        ht = HashTable(
            collision_resolution="linear_probing"
        )
        ht.put("apple", 10)
        ht.remove("apple")
        ht.put("apple", 20)
        self.assertEqual(ht.get("apple"), 20)

    def test_load_factor_calculation_with_deletions(self):
        """Test load factor calculation with deletions."""
        ht = HashTable(
            collision_resolution="linear_probing",
            initial_capacity=4,
            load_factor_threshold=0.6
        )
        ht.put("a", 1)
        ht.put("b", 2)
        ht.remove("a")
        ht.put("c", 3)
        self.assertGreater(ht.capacity, 4)


# Model Breaking Proof

#### Model Breaking Task URL: <https://labeling-ms.turing.com/conversations/93820/view>

#### Model code:

```python
# code generated by the model
class HashTable:
    """
    A hash table with support for two collision resolution strategies:
    'chaining' and 'linear_probing'. It resizes automatically when the
    load factor exceeds a user-defined threshold.
    """

    # A special singleton marker for deleted entries in linear probing
    _DELETED = object()

    def __init__(
        self,
        collision_resolution: str = 'chaining',
        initial_capacity: int = 8,
        load_factor_threshold: float = 0.75
    ):
        # Validate collision resolution
        if collision_resolution not in ('chaining', 'linear_probing'):
            raise ValueError("Collision resolution must be 'chaining' or 'linear_probing'")

        # Validate initial capacity
        if not isinstance(initial_capacity, int) or initial_capacity <= 0:
            raise ValueError("Initial capacity must be a positive integer")

        # Validate load factor threshold
        if (not isinstance(load_factor_threshold, float) or
                not (0 < load_factor_threshold < 1)):
            raise ValueError("Load factor threshold must be a float between 0 and 1 (exclusive)")

        self.collision_resolution = collision_resolution
        self.capacity = initial_capacity
        self.load_factor_threshold = load_factor_threshold
        self.size = 0  # Number of actual key-value pairs

        if self.collision_resolution == 'chaining':
            # Each index will hold a list of (key, value) pairs for chaining
            self.table = [[] for _ in range(self.capacity)]
        else:
            # For linear probing, each index can hold (key, value) or _DELETED or None
            self.table = [None] * self.capacity

    def _hash(self, key: str) -> int:
        # Simple modulus-based hashing
        return abs(hash(key)) % self.capacity

    def _needs_resize(self) -> bool:
        return (self.size / self.capacity) > self.load_factor_threshold

    def _resize(self) -> None:
        old_table = self.table
        old_capacity = self.capacity
        self.capacity *= 2  # Double the capacity
        self.size = 0  # We'll re-insert and recount

        if self.collision_resolution == 'chaining':
            # Reinitialize with empty chaining buckets
            self.table = [[] for _ in range(self.capacity)]

            # Re-insert existing items
            for bucket in old_table:
                for (k, v) in bucket:
                    self.put(k, v)
        else:
            # Reinitialize for linear probing
            self.table = [None] * self.capacity

            # Re-insert existing items (skip deleted or None entries)
            for entry in old_table:
                if entry and entry is not self._DELETED:
                    k, v = entry
                    self.put(k, v)

def put(self, key: str, value) -> None:
    # Validate key
    if not isinstance(key, str) or not key:
        raise TypeError("Key must be a non-empty string")

# Model breaking Proof

    # Issue: The issue here is with the `_needs_resize()` logic in `put()` function. it fails to consider that inserting a new item will certainly increase the size beyond the threshold, especially when capacity is small.

    # This is what actually causes the model code failure when testing  for resize chaining, resize linear_probing, and load_factor calculation with deletions. hence raising the `AssertionError: 4 not greater than 4`
  
    # Requirement Violated: "The table should be able to resize itself automatically when the number of items reaches a certain threshold or load factor."

    # Explanation: The check `if self._needs_resize():` in `put()` only verifies if the current load factor exceeds the threshold, before inserting the new item. When the size is just below the threshold, adding one more item may push it over, in this cases the code doesnt carryout the resizing in time, thus leading to incorrect behavior and the capacity not increasing when it should.

    # The best practice would have been to predict the post-insertion load factor which will greatly help to trigger resize correctly and in time.

    if (self.size + 1) / self.capacity > self.load_factor_threshold:
        self._resize()

        if self.collision_resolution == 'chaining':
            self._put_chaining(key, value)
        else:
            self._put_linear_probing(key, value)

    def _put_chaining(self, key: str, value) -> None:
        index = self._hash(key)
        bucket = self.table[index]
        # Check if the key already exists
        for i, (k, _) in enumerate(bucket):
            if k == key:
                raise KeyError(f"Key '{key}' already exists")
        # Insert new key-value
        bucket.append((key, value))
        self.size += 1

    def _put_linear_probing(self, key: str, value) -> None:
        index = self._hash(key)
        start_index = index

        while True:
            entry = self.table[index]
            if entry is None or entry is self._DELETED:
                # Empty or previously deleted slot found, insert new entry
                self.table[index] = (key, value)
                self.size += 1
                return
            else:
                existing_key, _ = entry
                if existing_key == key:
                    # Duplicate key found
                    raise KeyError(f"Key '{key}' already exists")

            index = (index + 1) % self.capacity
            # If we loop over the entire table, something is wrong (shouldn't happen if we resize)
            if index == start_index:
                raise RuntimeError("HashTable is full, cannot insert.")

    def get(self, key: str):
        # Validate key
        if not isinstance(key, str) or not key:
            raise TypeError("Key must be a non-empty string")

        if self.collision_resolution == 'chaining':
            return self._get_chaining(key)
        else:
            return self._get_linear_probing(key)

    def _get_chaining(self, key: str):
        index = self._hash(key)
        bucket = self.table[index]
        for (k, v) in bucket:
            if k == key:
                return v
        # If not found
        raise KeyError(f"Key '{key}' not found")

    def _get_linear_probing(self, key: str):
        index = self._hash(key)
        start_index = index

        while True:
            entry = self.table[index]
            if entry is None:
                # We hit an empty slot, key not found
                raise KeyError(f"Key '{key}' not found")
            elif entry is not self._DELETED:
                existing_key, existing_value = entry
                if existing_key == key:
                    return existing_value
            # Keep probing
            index = (index + 1) % self.capacity
            if index == start_index:
                break

        raise KeyError(f"Key '{key}' not found")

    def remove(self, key: str) -> None:
        # Validate key
        if not isinstance(key, str) or not key:
            raise TypeError("Key must be a non-empty string")

        if self.collision_resolution == 'chaining':
            self._remove_chaining(key)
        else:
            self._remove_linear_probing(key)

    def _remove_chaining(self, key: str) -> None:
        index = self._hash(key)
        bucket = self.table[index]
        for i, (k, _) in enumerate(bucket):
            if k == key:
                # Remove the (key, value) pair
                bucket.pop(i)
                self.size -= 1
                return
        raise KeyError(f"Key '{key}' not found")

    def _remove_linear_probing(self, key: str) -> None:
        index = self._hash(key)
        start_index = index

        while True:
            entry = self.table[index]
            if entry is None:
                # Not found
                raise KeyError(f"Key '{key}' not found")
            elif entry is not self._DELETED:
                existing_key, _ = entry
                if existing_key == key:
                    # Mark as deleted
                    self.table[index] = self._DELETED
                    self.size -= 1
                    return
            # Keep probing
            index = (index + 1) % self.capacity
            if index == start_index:
                break

        raise KeyError(f"Key '{key}' not found")

# <Issue>: <Add the issue in the model code here>

# code generated by the model
```